In [14]:

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.estimator import regression
import tflearn
import numpy as np
from tflearn.metrics import Accuracy

In [15]:
selected_df = pd.read_csv("ideology5_encoded.csv")
#selected_df = selected_df.drop("Unnamed: 0",axis=1)
ide = selected_df["IDEOLOGY5"].copy()
selected_df = selected_df.drop("IDEOLOGY5",axis=1)
for index, values in selected_df.iteritems():
    selected_df[index] = values / values.max()
selected_df["IDEOLOGY5"] = ide
selected_df

Unnamed: 0   HR6  MEDIA12  IDEOLOGY6  DEMOGRAPHIC4_IN  DEMOGRAPHIC4_CO  \
0       0.000000  0.75      0.5       0.50              1.0              0.0   
1       0.000826  0.50      0.5       0.50              0.0              1.0   
2       0.001653  0.00      0.0       0.00              0.0              0.0   
3       0.002479  0.50      1.0       0.75              0.0              0.0   
4       0.003306  1.00      0.5       0.50              0.0              0.0   
...          ...   ...      ...        ...              ...              ...   
1206    0.996694  0.75      0.0       0.25              0.0              0.0   
1207    0.997521  1.00      0.5       0.00              0.0              0.0   
1208    0.998347  0.50      0.0       1.00              0.0              0.0   
1209    0.999174  1.00      0.0       0.00              0.0              0.0   
1210    1.000000  0.25      0.0       0.75              0.0              0.0   

      DEMOGRAPHIC4_GA  DEMOGRAPHIC4_IL  DEMOGRAPHIC4_MS  DEMOGRAPHIC4_TX  ...  \
0                 0.0              0.0              0.0              0.0  ...   
1                 0.0              0.0              0.0              0.0  ...   
2                 1.0              0.0              0.0              0.0  ...   
3                 0.0              1.0              0.0              0.0  ...   
4                 0.0              0.0              1.0              0.0  ...   
...               ...              ...              ...              ...  ...   
1206              0.0              0.0              0.0              0.0  ...   
1207              0.0              0.0              0.0              0.0  ...   
1208              0.0              1.0              0.0              0.0  ...   
1209              0.0              0.0              0.0              0.0  ...   
1210              0.0              0.0              0.0              0.0  ...   

      DEMOGRAPHIC5_31323  DEMOGRAPHIC5_90005  DEMOGRAPHIC5_18051  \
0                    0.0                 0.0                 0.0   
1                    0.0                 0.0                 0.0   
2                    0.0                 0.0                 0.0   
3                    0.0                 0.0                 0.0   
4                    0.0                 0.0                 0.0   
...                  ...                 ...                 ...   
1206                 0.0                 0.0                 0.0   
1207                 0.0                 0.0                 0.0   
1208                 0.0                 0.0                 0.0   
1209                 0.0                 0.0                 0.0   
1210                 0.0                 0.0                 0.0   

      DEMOGRAPHIC5_25271  DEMOGRAPHIC5_43701  DEMOGRAPHIC5_34475  \
0                    0.0                 0.0                 0.0   
1                    0.0                 0.0                 0.0   
2                    0.0                 0.0                 0.0   
3                    0.0                 0.0                 0.0   
4                    0.0                 0.0                 0.0   
...                  ...                 ...                 ...   
1206                 0.0                 1.0                 0.0   
1207                 0.0                 0.0                 1.0   
1208                 0.0                 0.0                 0.0   
1209                 0.0                 0.0                 0.0   
1210                 0.0                 0.0                 0.0   

      DEMOGRAPHIC5_6152  DEMOGRAPHIC5_21740  DEMOGRAPHIC5_40353  IDEOLOGY5  
0                   0.0                 0.0                 0.0          2  
1                   0.0                 0.0                 0.0          0  
2                   0.0                 0.0                 0.0          2  
3                   0.0                 0.0                 0.0          0  
4                   0.0                 0.0          

In [16]:
encod = [[0,0,0,0,0],
        [1,0,0,0,0],
        [1,1,0,0,0],
        [1,1,1,0,0],
        [1,1,1,1,0],
        [1,1,1,1,1]]
tmp = []
values = [-2,-1,0,1,2]
for i in selected_df["IDEOLOGY5"].tolist():
    tmp.append(encod[values.index(i)])
selected_df["IDEOLOGY5"] = tmp
selected_df["IDEOLOGY5"].tolist()[0]

[1, 1, 1, 1, 0]

In [17]:
X = selected_df.drop("IDEOLOGY5",axis=1)
y = selected_df["IDEOLOGY5"]
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size = .25, random_state = 72)

In [18]:
x_train.shape

(908, 996)

In [19]:

x_train = np.array(x_train.drop("Unnamed: 0",axis=1))
x_test = np.array(x_test.drop("Unnamed: 0",axis=1))
x_train

array([[1.  , 0.5 , 1.  , ..., 0.  , 0.  , 0.  ],
       [0.5 , 1.  , 0.5 , ..., 0.  , 0.  , 0.  ],
       [1.  , 1.  , 0.75, ..., 0.  , 0.  , 0.  ],
       ...,
       [1.  , 1.  , 1.  , ..., 0.  , 0.  , 0.  ],
       [0.25, 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       [1.  , 1.  , 0.75, ..., 0.  , 0.  , 0.  ]])

In [20]:
out = np.array(y_train, dtype=object)
out = out.tolist()
out_test = np.array(y_test, dtype=object)
out_test = out_test.tolist()

In [22]:

tf.compat.v1.reset_default_graph()
acc = Accuracy()
model = input_data(shape=(None,995), name='input') # This basically just specifies the shape of input
model = fully_connected(model, 25, activation='relu')
model = fully_connected(model, 5, activation='sigmoid')
model = regression(model, optimizer='adam', learning_rate=1e-2, loss='categorical_crossentropy',metric = acc, name='target')
model = tflearn.DNN(model, tensorboard_dir='log')
# x_train = np.array([i[0] for i in x_train]).reshape(-1, 908, 996)
# y_train = np.array([i[1] for i in y_train]).reshape(-1, 1)
#out = np.empty_like(y_train, dtype=object)
model.fit(x_train,out, n_epoch = 100, validation_set=(x_test,out_test), shuffle = True, run_id = 'AlphaSnake.tflearn')
#model.evaluate(x_train, np.array(out))
#model.fit(x_train,y_train, n_epoch = 3, shuffle = True, run_id = 'AlphaSnake.tflearn')
#model.save('AlphaSnake.tflearn')
# x_test1 = np.array(x_test.drop("Unnamed: 0",axis=1))
# model.predict(x_test1)

Training Step: 1499  | total loss: 2.66212 | time: 0.091s
| Adam | epoch: 100 | loss: 2.66212 -- iter: 896/908
Training Step: 1500  | total loss: 2.64833 | time: 1.110s
| Adam | epoch: 100 | loss: 2.64833 | val_loss: 3.27997 -- iter: 908/908
--


In [23]:
out = np.array(y_test, dtype=object)
out = out.tolist()
model.evaluate(x_test,np.array(out_test))

[0.7623762411646323]

In [24]:
model.save('AlphaSnake.tflearn')

INFO:tensorflow:C:\Notebooks\DS\ReformedAI\AlphaSnake.tflearn is not in all_model_checkpoint_paths. Manually adding it.
